# Data Augmentation over Kunisch Patterns. 
## Seminario de Tesis I, Primavera 2022 
### MDS Program. University of Chile.
#### Supervisor: Prof. Benjamín Bustos, Prof. Iván Sipirán
#### Author: Matías Vergara

Performs data augmentation on patterns through the application of linear transformations.

## Imports

In [104]:
import cv2
import pandas as pd
from IPython.display import display
import os
import math
import random
import shutil

## Mounting Google Drive

In [105]:
# Mounting google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

## Define flags

It is enough to select ds flags and number of crops (this one will have effect depending on flags) and then run the rest of the cells. This way, two folders will be created: a labels one and a patterns one. Both of them will be named after the selected flags, separed by "_"

In [ ]:
DS_FLAGS = ['ref'] # ref, rot, crop, gaus
CROP_TIMES = 2

## Transformations

In [ ]:
data_flags = '_'.join(DS_FLAGS)

COPY_FLAGS = DS_FLAGS.copy()
if 'crop' in DS_FLAGS and len(DS_FLAGS) > 1:
    COPY_FLAGS.remove('crop')
    data_flags = '_'.join(COPY_FLAGS) + '_crop' + str(CROP_TIMES)
elif 'crop' in DS_FLAGS and len(DS_FLAGS) == 1:
    data_flags = 'crop' + str(CROP_TIMES)

In [ ]:
def rotate90(path):
    image = cv2.imread(path)
    rotated = cv2.rotate(image, cv2.cv2.ROTATE_90_CLOCKWISE)
    # cv2.imshow("90", rotated)
    return(rotated, "rot90")


def rotate180(path):
    image = cv2.imread(path)
    rotated = cv2.rotate(image, cv2.cv2.ROTATE_180)
    # cv2.imshow("180", rotated)
    return(rotated, "rot180")


def rotate270(path):
    image = cv2.imread(path)
    rotated = cv2.rotate(image, cv2.cv2.ROTATE_180)
    rotated = cv2.rotate(rotated, cv2.cv2.ROTATE_90_CLOCKWISE)
    # cv2.imshow("270", rotated)
    return (rotated, "rot270")


def invertX(path):
    image = cv2.imread(path)
    flipped = cv2.flip(image, 1)
    # cv2.imshow("flipX", flipped)
    return(flipped, "invX")


def invertY(path):
    image = cv2.imread(path)
    flipped = cv2.flip(image, 0)
    # cv2.imshow("flipY", flipped)
    return(flipped, "invY")


def crop(path, min_width = 1/2, min_height= 1/2, max_width = 1/1.1,
         max_height = 1/1.1):
    image = cv2.imread(path)
    height, width = image.shape[0], image.shape[1] # Caution: there are images in RGB and GS
    min_width = math.ceil(width * min_width)
    min_height = math.ceil(height * min_height)
    max_width = math.ceil(width * max_width)
    max_height = math.ceil(height * max_height)
    x1 = random.randint(0, width - min_width)
    w = random.randint(min_width, width - x1)
    y1 = random.randint(0, height - min_height)
    h = random.randint(min_height, height - y1)
    crop = image[y1:y1+h, x1:x1+w]
    return (crop, "crop")

def apply_transformations(pin, pout, transformations):
    # ../patterns/originals/84e/84e.png
    new_names = []
    i = 0
    for transformation in transformations:
        result, transf_name = transformation(pin)
        if transf_name == "crop": # special treatment for crops
          transf_name += str(i)
          i+=1
        path_els = pin.split("/")
        obj_name = path_els[3] + "_" + transf_name
        filename = obj_name + ".png"
        os.makedirs(pout, exist_ok = True)
        cv2.imwrite(pout + filename, result)
        new_names.append(obj_name)
    return new_names

# Select data augmentation functions based on data flags

MAP_FLAGS = {'ref': [invertX, invertY],
             'rot': [rotate90, rotate180, rotate270],
             'crop': [crop] * CROP_TIMES,
             'gaus': []
             }

ALLOWED_TRANSFORMATIONS = []
for f in DS_FLAGS:
    ALLOWED_TRANSFORMATIONS += MAP_FLAGS[f]
HOR_TRANSFORMATIONS = [invertX, rotate180] + [crop] * CROP_TIMES
VER_TRANSFORMATIONS = [invertY, rotate180] + [crop] * CROP_TIMES
COMMON_TRANSFORMATIONS = [invertX, invertY, rotate90, rotate180, rotate270] + [crop] * CROP_TIMES

for t in HOR_TRANSFORMATIONS:
    if t not in ALLOWED_TRANSFORMATIONS:
        HOR_TRANSFORMATIONS.remove(t)
print(HOR_TRANSFORMATIONS)

for t in VER_TRANSFORMATIONS:
    if t not in ALLOWED_TRANSFORMATIONS:
        VER_TRANSFORMATIONS.remove(t)
print(VER_TRANSFORMATIONS)

for t in COMMON_TRANSFORMATIONS:
    if t not in ALLOWED_TRANSFORMATIONS:
        COMMON_TRANSFORMATIONS.remove(t)
print(COMMON_TRANSFORMATIONS)

## Dataset splitting


In [ ]:
df = pd.read_json(folder_path + "labels/normalized_df.json", orient='index', encoding='latin-1')
classes = pd.read_csv(folder_path + "labels/class_labels.csv", index_col=0)

display(classes)

index = df.index.values
colnames = df.columns

valNumber = math.ceil(0.1 * len(index))
testNumber = math.ceil(0.1 * len(index))
trainNumber = len(index) - valNumber - testNumber

print("Patterns for training: {}".format(trainNumber))
print("Patterns for validation: {}".format(valNumber))
print("Patterns for testing: {}".format(testNumber))


random.shuffle(index)

elem_train = index[:trainNumber]
elem_val = index[trainNumber:trainNumber+valNumber]
elem_test = index[trainNumber+valNumber:]

assert (valNumber + testNumber + trainNumber) == len(index)

# print(elem_train)
# print(elem_val)
# print(elem_test)

## Augmentation
(Only over training set)



In [145]:
new_entries = {}

for pattern in elem_train: # only training set
    labels = df.loc[[pattern]]
    lbl_class = classes.loc[[pattern]]['chapter'].values[0]
    path_in = folder_path + "patterns/originals/" + pattern + "/" + pattern + ".png"
    path_out = folder_path + 'patterns/' + data_flags + '/train/' + lbl_class + "/"
    is_hor = labels['horizontal'].values[0]
    is_ver = labels['vertical'].values[0]
    if is_hor and is_ver:
        pass
    if is_hor and not is_ver:
        new_names = apply_transformations(path_in, path_out, HOR_TRANSFORMATIONS)
        labels = df.loc[[pattern]].values[0]
    elif is_ver and not is_hor:
        new_names = apply_transformations(path_in, path_out, VER_TRANSFORMATIONS)
        labels = df.loc[[pattern]].values[0]
    else: #if not is_hor and not is_ver:
        new_names = apply_transformations(path_in, path_out, COMMON_TRANSFORMATIONS)
        labels = df.loc[[pattern]].values[0]
    for name in new_names:
        new_entries[name] = labels
    # add the base pattern to the folder
    shutil.copy(path_in, path_out)

for pattern in elem_val:
    lbl_class = classes.loc[[pattern]]['chapter'].values[0]
    path_in = folder_path + "patterns/originals/" + pattern + "/" + pattern + ".png"
    path_out = folder_path + 'patterns/' + data_flags + '/val/' + lbl_class + "/"
    os.makedirs(path_out, exist_ok = True)
    shutil.copy(path_in, path_out)

for pattern in elem_test:
    lbl_class = classes.loc[[pattern]]['chapter'].values[0]
    path_in = folder_path + "patterns/originals/" + pattern + "/" + pattern + ".png"
    path_out = folder_path + 'patterns/' + data_flags + '/test/' + lbl_class + "/"
    os.makedirs(path_out, exist_ok = True)
    shutil.copy(path_in, path_out)

# agregar todas las entradas de elem_train a new_entries, y crear 
# el dataset "augmented_train_df.json"

for p in elem_train:
  labels = df.loc[p]
  new_entries[p] = labels.values

labels_output = folder_path + "labels/" + data_flags + "/"

os.makedirs(labels_output, exist_ok = True)

df_train = pd.DataFrame.from_dict(new_entries, columns=colnames, orient='index')
df_train.to_json(labels_output + "augmented_train_df.json", orient='index')

# agregar todas las entradas de elem_val a val_entries, y crear 
# el dataset "val_df.json"
val_entries = {}
for p in elem_val:
  labels = df.loc[p]
  val_entries[p] = labels.values

df_val = pd.DataFrame.from_dict(val_entries, columns=colnames, orient='index')
df_val.to_json(labels_output + "val_df.json", orient='index')

# agregar todas las entradas de elem_test a test_entries, y crear
# el dataset "test_df.json"
test_entries = {}
for p in elem_test:
  labels = df.loc[p]
  test_entries[p] = labels.values

df_test = pd.DataFrame.from_dict(test_entries, columns=colnames, orient='index')
df_test.to_json(labels_output + "test_df.json", orient='index')